In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import xarray as xr
import glob
import matplotlib.pyplot as plt
from rdflib import Graph, Namespace, URIRef, Literal
os.chdir("/workspaces/aqqa-kg-creation-dev/")
from src.utils import clip_netcdf_to_bb

In [3]:
year = "2020"
month = "01"
bounding_box = [45.82, 50.65, 8.95, 17.22]

In [5]:
ds_dict = {}

# read in data 
search_string = f"/mnt/data/raw/cams_euro_aq_reanalysis/{year}/download_{year}_{month}_all_variables/*"
path_to_files = glob.glob(search_string)

for path_to_file in path_to_files:

    # extract variable from path name
    var = path_to_file.split(".")[-4]

    # open dataset
    ds = xr.open_dataset(path_to_file)

    # clipt data to bounding box
    ds_bb = clip_netcdf_to_bb(ds, bounding_box)

    # period averaging
    if var == "O3":
        ds_bb_agg = ds_bb.resample(time="8H").mean()
    else:
        ds_bb_agg= ds_bb.resample(time="24H").mean()

    # save netcdf 
    base_path_output = f"/mnt/data/processed/CAMS_AQ_AOI_AGG/{year}/{month}"
    if not os.path.exists(base_path_output):
        os.makedirs(base_path_output)
    path_output = os.path.join(base_path_output, f"cams_aq_{var}_{year}{month}.nc")
    ds_bb_agg.to_netcdf(path_output)